In [1]:
import pandas as pd

In [2]:
import pyspark

In [4]:
from pyspark import SparkContext

#We Must a Start SparkSession

In [5]:
from pyspark.sql import SparkSession

In [7]:
Spark = SparkSession.builder.appName('Practise').getOrCreate()

In [8]:
Spark

In [9]:
df_pyspark = Spark.read.csv('data.csv')
df_pyspark
df_pyspark.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|      _c0|      _c1|                 _c2|     _c3|           _c4|      _c5|       _c6|           _c7|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.

#Getting the main column

In [10]:
df = Spark.read.option('header', 'true').csv('data.csv').show()
df

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [12]:
df = Spark.read.csv('data.csv', header =True, inferSchema =True)
df

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: int, Country: string]

In [ ]:
#df.drop('Country').show()
#df.describe().show()
#df.select(['Quantity','Country']).show()
#df.withColumn('Country_new', df['Country']).show()

In [13]:
df.createOrReplaceTempView('data')

In [14]:
results = Spark.sql("select * from data")
results.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [ ]:
#results.createOrReplaceTempView("still_data")

In [ ]:
#valsqlDF = Spark.sql("SELECT sum(Quantity * UnitPrice) as Total, Country FROM still_data group by Country");
#valsqlDF.show()

In [17]:
valsqlDF2 = Spark.sql("SELECT  Country, InvoiceNo, sum(Quantity * UnitPrice) as Total FROM data group by Country, InvoiceNo Order by Total desc");

In [16]:
valsqlDF2.show()

+--------------+---------+------------------+
|       Country|InvoiceNo|             Total|
+--------------+---------+------------------+
|United Kingdom|   581483|          168469.6|
|United Kingdom|   541431|           77183.6|
|United Kingdom|   574941| 52940.93999999999|
|United Kingdom|   576365|50653.909999999996|
|United Kingdom|   556444|           38970.0|
|United Kingdom|   567423|31698.159999999996|
|     Australia|   556917|22775.930000000008|
|United Kingdom|   572209|           22206.0|
|United Kingdom|   567381|           22104.8|
|     Australia|   563614|21880.439999999995|
|United Kingdom|   548203|          21627.72|
|United Kingdom|   550461|           21535.9|
|   Netherlands|   572035|          20277.92|
|   Netherlands|   563076| 19150.65999999999|
|United Kingdom|   562439|18841.480000000003|
|United Kingdom|   539750|18745.859999999997|
|United Kingdom|   573585|16874.579999999936|
|          EIRE|   541220|16774.719999999998|
|     Australia|   545475|        

In [20]:
valsqlDF2 = Spark.sql("SELECT  Country, InvoiceNo, sum(Quantity * UnitPrice) over(partition by Country order by InvoiceDate desc) as Total FROM data ");

In [21]:
valsqlDF2.show()

+---------+---------+-----+
|  Country|InvoiceNo|Total|
+---------+---------+-----+
|Australia|   565466|147.7|
|Australia|   565466|147.7|
|Australia|   565466|147.7|
|Australia|   565466|147.7|
|Australia|   565466|147.7|
|Australia|   565466|147.7|
|Australia|   565466|147.7|
|Australia|   565466|147.7|
|Australia|   568708|669.2|
|Australia|   568708|669.2|
|Australia|   568708|669.2|
|Australia|   568708|669.2|
|Australia|   568708|669.2|
|Australia|   568708|669.2|
|Australia|   568708|669.2|
|Australia|   568708|669.2|
|Australia|   568708|669.2|
|Australia|   568708|669.2|
|Australia|   568708|669.2|
|Australia|   568708|669.2|
+---------+---------+-----+
only showing top 20 rows

